# Setup

In [0]:
!pip install -U --pre tensorflow=="2.*"

Requirement already up-to-date: tensorflow==2.* in /usr/local/lib/python3.6/dist-packages (2.2.0)


In [0]:
!pip install pycocotools

In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [0]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1082204 sha256=014066fe8b09339c10c2299d5fad51aa1f3c0285f1b1a077324589b305e95275
  Stored in directory: /tmp/pip-ephem-wheel-cache-1hqv5wtu/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


## Imports 

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow.compat.v1 as tf
import zipfile
import cv2
import time


from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from google.colab.patches import cv2_imshow
from google.colab import files

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [0]:


def download_model(MODEL_NAME):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  #base_url = 'https://storage.cloud.google.com/mobilenet_edgetpu/checkpoints/'
  model_file = MODEL_NAME + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
      fname=MODEL_NAME,
      origin=base_url + model_file,
      untar=True,
      cache_dir='models/research/object_detection/')
  

Change model name here


In [0]:
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
#MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_NAME = 'ssd_inception_v2_coco_2018_01_28'

#!mkdir output1
#!mkdir output2
!mkdir output3

download_model(MODEL_NAME)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

mkdir: cannot create directory ‘output3’: File exists
278118400/278114232 [==============================] - 2s 0us/step


In [0]:
CWD_PATH = 'models/research/object_detection/datasets'
# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to video
PATH_TO_VIDEO = 'Pedestrian_Detect_2_1_1.mp4'


In [0]:
print(PATH_TO_CKPT)

models/research/object_detection/datasets/ssd_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [0]:
# Load the Tensorflow model into memory

start_time = time.time()
detection_graph = tf.Graph()

with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)
load_time = time.time()-start_time
print(load_time)

1.832366704940796


In [0]:
# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [0]:
video = cv2.VideoCapture(PATH_TO_VIDEO)

In [0]:
inference_time = []
count = 0

In [0]:


while(video.isOpened()):

    # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    ret, frame = video.read()
    if not ret:
      break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_expanded = np.expand_dims(frame_rgb, axis=0)

    # Perform the actual detection by running the model with the image as input
    start_time = time.time()
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})
    #print(boxes)
    inference_time.append(time.time()-start_time)
    #print(classes)
    boxes = np.squeeze(boxes)
    scores = np.squeeze(scores)
    classes = np.squeeze(classes).astype(np.int32)

    people_indices = np.argwhere(classes == 1)
    #print(people_indices)

    boxes = np.squeeze(boxes[people_indices], axis =1)
    scores = np.squeeze(scores[people_indices], axis =1)
    #print(scores)
    
    classes = np.squeeze(classes[people_indices], axis =1)

    
    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.array([boxes[0,:]]),
        np.array([classes[0]]),
        np.array([scores[0]]),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.18)

    # All the results have been drawn on the frame, so it's time to display it.
    #cv2_imshow(frame)
    #cv2.imwrite("output1/frame%d.jpg" % count,frame)
    #cv2.imwrite("output2/frame%d.jpg" % count,frame)
    cv2.imwrite("output3/frame%d.jpg" % count,frame)
    print(count)
    count +=1
    
    
    

    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break

# Clean up
video.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
print(np.average(inference_time))

0.23501875616043508


In [0]:
#f = open("output1/"+MODEL_NAME+".txt", "a")
#f = open("output2/"+MODEL_NAME+".txt", "a")
f = open("output3/"+MODEL_NAME+".txt", "a")
f.write("Load time: %f\n" % load_time)
f.write("Inference time: %f\n" % np.average(inference_time))
f.close()

In [0]:
#!zip -r output1.zip output1
#!zip -r output2.zip output2
!zip -r output3.zip output3

  adding: output3/ (stored 0%)
  adding: output3/frame1033.jpg (deflated 1%)
  adding: output3/frame1127.jpg (deflated 1%)
  adding: output3/frame1090.jpg (deflated 1%)
  adding: output3/frame1103.jpg (deflated 1%)
  adding: output3/frame636.jpg (deflated 1%)
  adding: output3/frame533.jpg (deflated 1%)
  adding: output3/frame280.jpg (deflated 1%)
  adding: output3/frame751.jpg (deflated 1%)
  adding: output3/frame320.jpg (deflated 2%)
  adding: output3/frame1031.jpg (deflated 1%)
  adding: output3/frame803.jpg (deflated 1%)
  adding: output3/frame937.jpg (deflated 1%)
  adding: output3/frame1354.jpg (deflated 2%)
  adding: output3/frame167.jpg (deflated 1%)
  adding: output3/frame1175.jpg (deflated 1%)
  adding: output3/frame359.jpg (deflated 1%)
  adding: output3/frame232.jpg (deflated 1%)
  adding: output3/frame1294.jpg (deflated 1%)
  adding: output3/frame154.jpg (deflated 1%)
  adding: output3/ssd_inception_v2_coco_2018_01_28.txt (deflated 7%)
  adding: output3/frame1049.jpg (defl

In [0]:
#files.download("output1.zip")
#files.download("output2.zip")
#files.download("output3.zip")